In [1]:
import pandas as pd

In [3]:
df=pd.read_csv('../2_data/updated_category.csv')
df.head()

,id,Question,Answer,Category
0,0,What are some examples of breed-typical person...,Breed-typical personalities in dogs are develo...,{'Dog'}
1,1,How do working and herding dogs typically behave?,Working and herding dogs have business-like di...,{'Dog'}
2,2,Which breed of dogs is known for strong loyalty?,Collies and Akitas are known for their strong ...,{'Dog'}
3,3,What instincts do guarding dogs typically disp...,Guarding dogs tend to be protective of their t...,{'Dog'}
4,4,How does breed specificity impact a dog's abil...,Breed specificity affects how well dogs adapt ...,{'Dog'}


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        583 non-null    int64 
 1   Question  583 non-null    object
 2   Answer    583 non-null    object
 3   Category  583 non-null    object
dtypes: int64(1), object(3)
memory usage: 18.3+ KB


In [5]:
df.columns

Index(['id', 'Question', 'Answer', 'Category'], dtype='object')

In [9]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py


--2024-09-09 12:21:52--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: 'minsearch.py'

     0K ...                                                   100% 12.1M=0s

2024-09-09 12:21:52 (12.1 MB/s) - 'minsearch.py' saved [3832/3832]



In [6]:
import minsearch


In [7]:

documents = df.to_dict(orient='records')


In [8]:
documents[0]

{'id': 0,
 'Question': 'What are some examples of breed-typical personalities in dogs?',
 'Answer': 'Breed-typical personalities in dogs are developed through generations of selection for certain traits. For example, sporting dogs are adventurous and responsive to humans, while hounds tend to be more independent and aloof.',
 'Category': "{'Dog'}"}

In [9]:
index = minsearch.Index(
     text_fields=["Question", "Answer","Category"],
     keyword_fields=['id']
)

In [10]:
index.fit(documents)


## RAG flow

In [11]:
from openai import OpenAI
client = OpenAI()


In [12]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [14]:
query = 'which dog breed is the most friendly?'


In [15]:
search_results=search(query)
print(search_results, len(search_results))

[{'id': 2, 'Question': 'Which breed of dogs is known for strong loyalty?', 'Answer': 'Collies and Akitas are known for their strong sense of loyalty.', 'Category': "{'Dog'}"}, {'id': 517, 'Question': 'How can I choose the right dog or breed for my lifestyle?', 'Answer': "Consider factors such as the dog's size, energy level, temperament, and grooming needs. Visiting potential dog candidates in person is also recommended.", 'Category': "{'Dog'}"}, {'id': 4, 'Question': "How does breed specificity impact a dog's ability to adapt to new surroundings or owners?", 'Answer': 'Breed specificity affects how well dogs adapt to new environments or owners. Instinctive behaviors, such as guarding or herding, are often breed-specific and cannot be taught. Mixed breeds may also exhibit unique instincts.', 'Category': "{'Dog'}"}, {'id': 456, 'Question': ' Why has most research on dog cognition focused on pet dogs in human homes?', 'Answer': ' Most research on dog cognition has focused on pet dogs in 

In [16]:
from openai import OpenAI
client = OpenAI()


In [17]:
prompt_template = """
You are a knowledgeable pet care expert specializing in dogs and cats. 
Answer the USER QUESTION based on the INFORMATION from our pet care database. 
Use only the facts from the INFORMATION when answering the USER QUESTION. 
If the information doesn't provide a complete answer, 
say so and suggest seeking professional veterinary advice.

USER QUESTION: {question}

INFORMATION:
{context}
""".strip()

entry_template = """
Question: {Question}
Answer: {Answer}
Category: {Category}
""".strip()


def build_pet_care_prompt(query, search_results): 
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc)+"\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [18]:
print(build_pet_care_prompt(query, search_results))

You are a knowledgeable pet care expert specializing in dogs and cats. 
Answer the USER QUESTION based on the INFORMATION from our pet care database. 
Use only the facts from the INFORMATION when answering the USER QUESTION. 
If the information doesn't provide a complete answer, 
say so and suggest seeking professional veterinary advice.

USER QUESTION: which dog breed is the most friendly?

INFORMATION:
Question: Which breed of dogs is known for strong loyalty?
Answer: Collies and Akitas are known for their strong sense of loyalty.
Category: {'Dog'}

Question: How can I choose the right dog or breed for my lifestyle?
Answer: Consider factors such as the dog's size, energy level, temperament, and grooming needs. Visiting potential dog candidates in person is also recommended.
Category: {'Dog'}

Question: How does breed specificity impact a dog's ability to adapt to new surroundings or owners?
Answer: Breed specificity affects how well dogs adapt to new environments or owners. Instincti

In [19]:
def llm(prompt,model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [20]:
len(documents)

583

In [21]:

def rag(query,model='gpt-4o-mini'):
    search_results = search(query)
    prompt = build_pet_care_prompt(query, search_results)
    answer = llm(prompt,model=model)
    return answer

In [22]:
query = 'What traits define the personality of sporting dog breeds'
answer=rag(query)
print(answer)


Sporting dog breeds are characterized by specific personality traits that set them apart. They are typically adventurous and responsive to humans. These traits have developed through generations of breeding for particular characteristics in sporting activities. This responsiveness often makes them eager to learn and participate in activities with their owners. If you have further questions about individual sporting breeds or their behaviors, it's always a good idea to consult with a professional veterinarian or a dog behavior specialist for tailored advice.


In [56]:
#rename first column to 'id'


In [23]:
df.head()

,id,Question,Answer,Category
0,0,What are some examples of breed-typical person...,Breed-typical personalities in dogs are develo...,{'Dog'}
1,1,How do working and herding dogs typically behave?,Working and herding dogs have business-like di...,{'Dog'}
2,2,Which breed of dogs is known for strong loyalty?,Collies and Akitas are known for their strong ...,{'Dog'}
3,3,What instincts do guarding dogs typically disp...,Guarding dogs tend to be protective of their t...,{'Dog'}
4,4,How does breed specificity impact a dog's abil...,Breed specificity affects how well dogs adapt ...,{'Dog'}


In [58]:
# update df back to file


In [28]:
# df=pd.read_csv('Dog-Cat-QA.csv')

In [24]:
df.head(),len(df)

(   id                                           Question  \
 0   0  What are some examples of breed-typical person...   
 1   1  How do working and herding dogs typically behave?   
 2   2   Which breed of dogs is known for strong loyalty?   
 3   3  What instincts do guarding dogs typically disp...   
 4   4  How does breed specificity impact a dog's abil...   
 
                                               Answer Category  
 0  Breed-typical personalities in dogs are develo...  {'Dog'}  
 1  Working and herding dogs have business-like di...  {'Dog'}  
 2  Collies and Akitas are known for their strong ...  {'Dog'}  
 3  Guarding dogs tend to be protective of their t...  {'Dog'}  
 4  Breed specificity affects how well dogs adapt ...  {'Dog'}  ,
 583)

In [25]:
query="What are the signs of stress in cats and dogs?"
answer=rag(query)
print(answer)


Signs of stress in dogs may include excessive yawning, lip-licking, trembling, avoiding or hiding, and a hardening of the eyes. 

For cats, the provided information does not specify the signs of stress. It would be advisable to consult a veterinarian to gain a better understanding of signs of stress in cats.


## Retrieval evaluation 

In [27]:
df_questions=pd.read_csv('../2_data/ground-truth-retrieval.csv')  
df_questions.head()

,id,question
0,0,What characteristics define sporting dog perso...
1,0,How do hounds typically behave in terms of per...
2,0,What influences the personalities of different...
3,0,In what ways do breed-specific traits affect d...
4,0,Can you explain the typical temperaments of sp...


In [28]:
len(df_questions)

2915

In [29]:
ground_truth = df_questions.to_dict(orient='records')


In [30]:
ground_truth[0]


{'id': 0,
 'question': 'What characteristics define sporting dog personalities?'}

In [31]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [32]:
def minsearch_search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [33]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [34]:
from tqdm.auto import tqdm

In [35]:
evaluate(ground_truth, lambda q: minsearch_search(q['question'] ))


  0%|          | 0/2915 [00:00<?, ?it/s]

{'hit_rate': 0.8977701543739279, 'mrr': 0.6993683465381566}

### Finding the best parameters


In [36]:
df_validation=df_questions[:100]
df_test=df_questions[100:]

In [37]:
import random

def simple_optimize(param_ranges, objective_function, n_iterations=10):
    best_params = None
    best_score = float('-inf')  # Assuming we're minimizing. Use float('-inf') if maximizing.

    for _ in range(n_iterations):
        # Generate random parameters
        current_params = {}
        for param, (min_val, max_val) in param_ranges.items():
            if isinstance(min_val, int) and isinstance(max_val, int):
                current_params[param] = random.randint(min_val, max_val)
            else:
                current_params[param] = random.uniform(min_val, max_val)
        
        # Evaluate the objective function
        current_score = objective_function(current_params)
        
        # Update best if current is better
        if current_score > best_score:  # Change to > if maximizing
            best_score = current_score
            best_params = current_params
    
    return best_params, best_score

In [38]:
gt_val = df_validation.to_dict(orient='records')


In [39]:
def minsearch_search(query, boost=None):
    if boost is None:
        boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [40]:
param_ranges = {
    'Question': (0.0, 3.0),
    'Answer': (0.0, 3.0),
    'Category': (0.0, 3.0),
}

def objective(boost_params):
    def search_function(q):
        return minsearch_search(q['question'], boost_params)

    results = evaluate(gt_val, search_function)
    return results['mrr']

In [41]:
simple_optimize(param_ranges, objective, n_iterations=20)


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

({'Question': 0.9367319821980193,
  'Answer': 2.3888355405240285,
  'Category': 2.1628814828290253},
 0.7538333333333332)

In [42]:
def minsearch_improved(query):
    boost = {
        'Question': 0.9367319821980193,
        'Answer': 2.3888355405240285,
        'Category': 2.1628814828290253,
    }

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

evaluate(ground_truth, lambda q: minsearch_improved(q['question']))

  0%|          | 0/2915 [00:00<?, ?it/s]

{'hit_rate': 0.9080617495711836, 'mrr': 0.7256326063873232}

### RAG evaluation

In [43]:
prompt2_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [44]:
len(ground_truth)


2915

In [45]:
record = ground_truth[0]


In [46]:
import json

In [47]:
df_sample = df_questions.sample(n=200, random_state=1)
df_sample


,id,question
1826,365,What health risks do fleas and ticks pose to c...
616,123,Do strawberries provide any nutritional value ...
1966,393,What male names are popular for kittens?
2813,562,How can I create a shopping list for a new puppy?
600,120,Can cats taste sweet fruits like strawberries ...
...,...,...
2379,475,What is the key facial expression of a Japanes...
1297,259,What type of clippers should I use for trimmin...
2904,580,What should I avoid to prevent stressing my se...
650,130,What steps should I take to incorporate probio...


In [48]:
sample= df_sample.to_dict(orient='records')

In [49]:
type(sample),len(sample)

(list, 200)

In [50]:
evaluations = []

for record in tqdm(sample):
    question = record['question']
    answer_llm = rag(question) 

    prompt = prompt2_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt)
    evaluation = json.loads(evaluation)

    evaluations.append((record, answer_llm, evaluation))

  0%|          | 0/200 [00:00<?, ?it/s]

In [51]:
df_eval = pd.DataFrame(evaluations, columns=['record', 'answer', 'evaluation'])

df_eval['id'] = df_eval.record.apply(lambda d: d['id'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])

del df_eval['record']
del df_eval['evaluation']

In [52]:
df_eval.relevance.value_counts(normalize=True)


relevance
RELEVANT           0.925
PARTLY_RELEVANT    0.065
NON_RELEVANT       0.010
Name: proportion, dtype: float64

In [60]:
df_eval_1=pd.read_csv('../2_data/rag-eval-gpt-4o-mini.csv')

In [61]:
df_eval_1.relevance.value_counts(normalize=True)


relevance
RELEVANT           0.935
PARTLY_RELEVANT    0.050
NON_RELEVANT       0.015
Name: proportion, dtype: float64

In [54]:
df_eval.to_csv('../2_data/rag-eval-gpt-4o-mini_1004.csv', index=False)


In [62]:
df_eval[df_eval.relevance == 'NON_RELEVANT']


,answer,id,question,relevance,explanation
97,"The information provided does not define ""cat-...",510,What are cat-sized openings and how can they b...,NON_RELEVANT,The generated answer does not address the ques...
136,The information provided does not contain any ...,444,What factors influence wolves' defense of terr...,NON_RELEVANT,The generated answer explicitly states that it...


In [63]:
df[df.id == 510]

,id,Question,Answer,Category
510,510,How can homeowners with creative thinking addr...,Homeowners with creative thinking can address ...,{'Dog'}
